In [ ]:
# install all libs
!pip install -r requirements.txt

In [9]:
import dotenv
# Reload the variables in your '.env' file (override the existing variables)
dotenv.load_dotenv(".env", override=True)

True

# GPT4 with Retrieval Augmentation over Web

In this notebook we'll work through an example of using GPT-4 with retrieval augmentation to answer questions about NOUS Wissensmanagement GmbH.

In this example, we will download all the content from https://www.nousdigital.net/en/. 

This downloads all HTML into the `nous` directory.

In [10]:
# crawl website content using wget
# !wget -e robots=off -r -A.html -P nous https://www.nousdigital.net/en/

Now we can use LangChain itself to process these docs. 
We do this using the `DirectoryLoader` together with the `BSHTMLLoader:

In [11]:
from langchain.document_loaders import BSHTMLLoader, DirectoryLoader

loader = DirectoryLoader('nous/www.nousdigital.net/en', loader_cls=BSHTMLLoader)

docs = loader.load()
len(docs)

232

This leaves us with `232` processed doc pages. Let's take a look at the format each one contains:

In [12]:
docs[20].page_content

'\n\n\n\n\n\n\n\n\n\n\n      NOUS solutions for culture\n      – NOUSdigital\n    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nData protection\n\n  \nThis website uses cookies and services in the categories listed below. Select REQUIRED ONLY to accept only technically necessary cookies or SAVE CHOICE to save your individual settings.\n\n\n      FURTHER INFORMATION\n    \n\nPrivacy Policy\n\nLegal Notice\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRequired Cookies\n\n\n\n\n\n\nThese cookies are needed to enable the basic functionality of this website. These cookies can therefore not be disabled.\n\n\n\n\nHTTP Cookie:\naccepted_optional_cookies\n\nPurpose: This cookie stores information about which optional cookies have been accepted or rejected.\nDomain: www.nousdigital.net\nStorage duration: 1 year\n\nThird party: No\n\n\n\n\n\nHTTP Cookie:\ncsrftoken\n\nPurpose: Protect against "Cross Site Request Forgery (CSRF)" attacks via form submission.\nDomain: www.nousdigital.net\nStorage duration: 1 year\n\

In [13]:
#remove '\n' in all docs.page_content
for doc in docs:
    doc.page_content = doc.page_content.replace('\n', ' ')

Now let's see how we can process all of these. We will chunk everything into ~500 token chunks, we can do this easily with `langchain` and `tiktoken`:

In [14]:
import tiktoken

tokenizer_name = tiktoken.encoding_for_model('gpt-4')
tokenizer_name.name

'cl100k_base'

In [15]:
tokenizer = tiktoken.get_encoding(tokenizer_name.name)

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

Process the `docs` into more chunks using this approach.

In [17]:
from typing_extensions import Concatenate
from uuid import uuid4
from tqdm.auto import tqdm

chunks = []

for idx, page in enumerate(tqdm(docs)):
    content = page.page_content
    if len(content) > 100:
        url = page.metadata['source'].replace('rtdocs/', 'https://')
        texts = text_splitter.split_text(content)
        chunks.extend([{
            'id': str(uuid4()),
            'text': texts[i],
            'chunk': i,
            'url': url
        } for i in range(len(texts))])

  0%|          | 0/232 [00:00<?, ?it/s]

In [18]:
chunks[2]

{'id': '6f66b0b4-0414-4acc-bb3d-b1eacb15243d',
 'text': 'I read the Privacy Policy and agree.                                                     SUBMIT      \xa0       Burger pink Copy                                         Home                                   What We Do                                   Our Products                                   Our Showroom                                   About us                                   Blog                                        Jobs',
 'chunk': 2,
 'url': 'nous/www.nousdigital.net/en/index.html'}

Our chunks are ready so now we move onto embedding and indexing everything.

## Initialize Embedding Model

We use `text-embedding-ada-002` as the embedding model. We can embed text like so:

In [19]:
import os
import openai


openai.api_key = os.getenv("OPENAI_API_KEY") or "OPENAI_API_KEY"

openai.Engine.list()  # verify that we are authenticated

<OpenAIObject list at 0x7f5f04845530> JSON: {
  "object": "list",
  "data": [
    {
      "object": "engine",
      "id": "whisper-1",
      "ready": true,
      "owner": "openai-internal",
      "permissions": null,
      "created": null
    },
    {
      "object": "engine",
      "id": "babbage",
      "ready": true,
      "owner": "openai",
      "permissions": null,
      "created": null
    },
    {
      "object": "engine",
      "id": "davinci",
      "ready": true,
      "owner": "openai",
      "permissions": null,
      "created": null
    },
    {
      "object": "engine",
      "id": "text-davinci-edit-001",
      "ready": true,
      "owner": "openai",
      "permissions": null,
      "created": null
    },
    {
      "object": "engine",
      "id": "babbage-code-search-code",
      "ready": true,
      "owner": "openai-dev",
      "permissions": null,
      "created": null
    },
    {
      "object": "engine",
      "id": "text-similarity-babbage-001",
      "ready": t

In [20]:
embed_model = "text-embedding-ada-002"

res = openai.Embedding.create(
    input=[
        "Sample document text goes here",
        "there will be several phrases in each batch"
    ], engine=embed_model
)

In the response `res` we will find a JSON-like object containing our new embeddings within the `'data'` field.

In [21]:
res.keys()

dict_keys(['object', 'data', 'model', 'usage'])

Inside `'data'` we will find two records, one for each of the two sentences we just embedded. Each vector embedding contains `1536` dimensions (the output dimensionality of the `text-embedding-ada-002` model.

In [22]:
len(res['data'])

2

In [23]:
len(res['data'][0]['embedding']), len(res['data'][1]['embedding'])

(1536, 1536)

We will apply this same embedding logic to the langchain docs dataset we've just scraped. But before doing so we must create a place to store the embeddings.

## Initializing the Index

Now we need a place to store these embeddings and enable a efficient vector search through them all. To do that we use Pinecone, we can get a [free API key](https://app.pinecone.io/) and enter it below where we will initialize our connection to Pinecone and create a new index.

In [24]:
import pinecone
import os

# setup pinecone account to get credentials
PINECONE_API_KEY=  os.getenv('PINECONE_API_KEY') or 'PINECONE_API_KEY'
PINECONE_ENVIRONMENT='us-west1-gcp-free'

os.environ['PINECONE_ENVIRONMENT'] = PINECONE_ENVIRONMENT

pinecone.init(api_key=PINECONE_API_KEY, enviroment=PINECONE_ENVIRONMENT)
pinecone.whoami()

WhoAmIResponse(username='a9d81a4', user_label='default', projectname='cc97d47')

In [25]:
index_name = 'gpt-4-nous-docs'

In [26]:
import time

# check if index already exists (it shouldn't if this is first time)
if index_name not in pinecone.list_indexes():
    # if does not exist, create index
    pinecone.create_index(
        index_name,
        dimension=len(res['data'][0]['embedding']),
        metric='cosine'
    )
    # wait for index to be initialized
    time.sleep(1)

# connect to index
index = pinecone.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1628}},
 'total_vector_count': 1628}

We can see the index is currently empty with a `total_vector_count` of `0`. We can begin populating it with OpenAI `text-embedding-ada-002` built embeddings like so:

In [27]:
chunks[1]['text']

'HTTP Cookie: _ga  Purpose: Used to distinguish users. Domain: www.nousdigital.net Storage duration: 2 years  Third party: Yes      HTTP Cookie: _gat  Purpose: Used to throttle the request rate. Domain: www.nousdigital.net Storage duration: Session  Third party: Yes      HTTP Cookie: _gid  Purpose: Registers a unique ID that is used to generate statistical data on how the visitor uses the website. Domain: www.nousdigital.net Storage duration: 23 hours  Third party: Yes        Service name: Google Maps   Privacy policy: https://policies.google.com/privacy   Owner: Google LLC       HTTP Cookie: _ga  Purpose: Used to distinguish users. Domain: www.nousdigital.net Storage duration: 2 years  Third party: Yes      HTTP Cookie: _gat  Purpose: Used to throttle the request rate. Domain: www.nousdigital.net Storage duration: Session  Third party: Yes      HTTP Cookie: _gid  Purpose: Registers a unique ID that is used to generate statistical data on how the visitor uses the website. Domain: www.n

In [28]:
openai.Embedding.create(input=chunks[1]['text'], engine=embed_model)

<OpenAIObject list at 0x7f5f188a74c0> JSON: {
  "object": "list",
  "data": [
    {
      "object": "embedding",
      "index": 0,
      "embedding": [
        -0.011502924375236034,
        0.0040663848631083965,
        0.0324503555893898,
        -0.010292090475559235,
        -0.004328732378780842,
        -0.018606485798954964,
        -0.0058086407370865345,
        -0.016063733026385307,
        -0.002013011835515499,
        -0.024068692699074745,
        0.002667198656126857,
        0.004073111806064844,
        0.010991684161126614,
        -0.02326147072017193,
        0.0030926724430173635,
        0.018512308597564697,
        0.025037359446287155,
        -0.023584358394145966,
        0.010742790065705776,
        -0.012747392989695072,
        -0.009182158857584,
        0.0032204824965447187,
        -0.0384238064289093,
        0.008233672939240932,
        -0.02137795090675354,
        -0.013016467913985252,
        0.004739070311188698,
        -0.01977695897221565

In [29]:
from tqdm.auto import tqdm
from time import sleep

batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(chunks), batch_size)):
    # find end of batch
    i_end = min(len(chunks), i+batch_size)
    meta_batch = chunks[i:i_end]
    # get ids
    ids_batch = [x['id'] for x in meta_batch]
    # get texts to encode
    texts = [x['text'] for x in meta_batch]
    # create embeddings (try-except added to avoid RateLimitError)
    try:
        res = openai.Embedding.create(input=texts, engine=embed_model)
    except Exception as e:
        print(e)
        done = False
        while not done:
            sleep(5)
            try:
                res = openai.Embedding.create(input=texts, engine=embed_model)
                done = True
            except:
                pass
    embeds = [record['embedding'] for record in res['data']]
    # cleanup metadata
    meta_batch = [{
        'text': x['text'],
        'chunk': x['chunk'],
        'url': x['url']
    } for x in meta_batch]
    to_upsert = list(zip(ids_batch, embeds, meta_batch))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

  0%|          | 0/17 [00:00<?, ?it/s]

Now we've added all of our langchain docs to the index. With that we can move on to retrieval and then answer generation using GPT-4.

## Retrieval

To search through our documents we first need to create a query vector `xq`. Using `xq` we will retrieve the most relevant chunks from the LangChain docs, like so:

In [30]:
def create_embedding(query):
    return openai.Embedding.create(
        input=[query],
        engine=embed_model
    )

def query_pinecone_embedding(openai_embedding):
    # retrieve from Pinecone
    xq = openai_embedding['data'][0]['embedding']
    # get relevant contexts (including the questions)
    return index.query(xq, top_k=5, include_metadata=True)

In [31]:
query = "Who is the CEO of NOUS Wissensmanagement GmbH?"

embedding = create_embedding(query)
result = query_pinecone_embedding(embedding)
result

{'matches': [{'id': '84bef6c5-3c1b-49a1-8b58-2275472d58d6',
              'metadata': {'chunk': 10.0,
                           'text': '3                           NOUS '
                                   'Wissensmanagement GmbH            '
                                   'Ullmannstraße 35 |           1150 Vienna, '
                                   'Austria |           '
                                   'info@nousdigital.com    Newsletter '
                                   'signup            '
                                   'register                                   '
                                   'Home                                       '
                                   'What We '
                                   'Do                                       '
                                   'Our '
                                   'Products                                       '
                                   'Our '
                                

With retrieval complete, we move on to feeding these into GPT-4 to produce answers.

## Retrieval Augmented Generation

GPT-4 is currently accessed via the `ChatCompletions` endpoint of OpenAI. To add the information we retrieved into the model, we need to pass it into our user prompts *alongside* our original query. We can do that like so:

In [32]:
def pinecone_to_list(res):
    # get list of retrieved texts
    contexts = [item['metadata']['text'] for item in res['matches']]
    return "\n\n---\n\n".join(contexts)+"\n\n-----\n\n"+query

pinecone_to_list(result)


'3                           NOUS Wissensmanagement GmbH            Ullmannstraße 35 |           1150 Vienna, Austria |           info@nousdigital.com    Newsletter signup            register                                   Home                                       What We Do                                       Our Products                                       Our Showroom                                       About us                                       Blog                      Legal Notice   Terms & Conditions   Privacy\n\n---\n\n3                           NOUS Wissensmanagement GmbH            Ullmannstraße 35 |           1150 Vienna, Austria |           info@nousdigital.com    Newsletter signup            register                                   Home                                       What We Do                                       Our Products                                       Our Showroom                                       About us                          

Now we configure following prompt:

In [33]:
def query_gpt(prompt, message, model="gpt-4"):
     return openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": message}
        ]
    )

In [34]:
# system message to 'prime' the model
prompt = f"""You are Q&A bot. A highly intelligent system that answers
user questions based on the information provided by the user above
each question. If the information can not be found in the information
provided by the user you truthfully say "I don't know".
"""

In [35]:
def search_docs(query, model="gpt-4"):
    embedding = create_embedding(query)
    result = query_pinecone_embedding(embedding)
    open_ai_message = pinecone_to_list(result)
    return query_gpt(prompt, open_ai_message, model=model)

In [46]:
query = "What services does NOUS provide?"
# "What can you tell me about the company NOUS?"
# "What services does NOUS provide?"
# "Who are the CEOs?"
# "Who is the CEO of NOUS Wissensmanagement?"
# "Which technologies does NOUS use?"

result = search_docs(query, model="gpt-4")
result

<OpenAIObject chat.completion id=chatcmpl-7R07k5hTpCw45CL58HwuIYZ5pPh7P at 0x7f5f048452b0> JSON: {
  "id": "chatcmpl-7R07k5hTpCw45CL58HwuIYZ5pPh7P",
  "object": "chat.completion",
  "created": 1686669688,
  "model": "gpt-4-0314",
  "usage": {
    "prompt_tokens": 728,
    "completion_tokens": 72,
    "total_tokens": 800
  },
  "choices": [
    {
      "message": {
        "role": "assistant",
        "content": "NOUS provides services in the field of mobile guides and media-based education. They advise institutions and companies on adapting their educational role for the digital world, and develop multimedia, intuitive end-user, and streaming applications on the web and other platforms. They have worked on over 200 projects of different sizes and focuses with museums, cultural organizations, and big brands worldwide."
      },
      "finish_reason": "stop",
      "index": 0
    }
  ]
}

In [37]:
result

<OpenAIObject chat.completion id=chatcmpl-7Qzefml3qq5Hu22RfDqnFSnbfPbTA at 0x7f5f04846840> JSON: {
  "id": "chatcmpl-7Qzefml3qq5Hu22RfDqnFSnbfPbTA",
  "object": "chat.completion",
  "created": 1686667885,
  "model": "gpt-4-0314",
  "usage": {
    "prompt_tokens": 728,
    "completion_tokens": 74,
    "total_tokens": 802
  },
  "choices": [
    {
      "message": {
        "role": "assistant",
        "content": "NOUS provides services in the field of mobile guides and media-based education. They advise institutions and companies on adapting their educational role to align with the digital world and develop multimedia, intuitive end-user, and streaming applications on the web and other platforms. They also create tailor-made concepts with innovative technology for various projects, partnering with museums, cultural organizations, and big brands worldwide."
      },
      "finish_reason": "stop",
      "index": 0
    }
  ]
}

To display this response nicely, we will display it in markdown.

In [38]:
from IPython.display import Markdown

display(Markdown(result['choices'][0]['message']['content']))

NOUS provides services in the field of mobile guides and media-based education. They advise institutions and companies on adapting their educational role to align with the digital world and develop multimedia, intuitive end-user, and streaming applications on the web and other platforms. They also create tailor-made concepts with innovative technology for various projects, partnering with museums, cultural organizations, and big brands worldwide.

Let's compare this to a non-augmented query...

In [39]:
res = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": prompt },
        {"role": "user", "content": query}
    ]
)
display(Markdown(res['choices'][0]['message']['content']))

I don't know.

If we drop the `"I don't know"` part of the `primer`?

In [40]:
res = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are Q&A bot. A highly intelligent system that answers user questions"},
        {"role": "user", "content": query}
    ]
)
display(Markdown(res['choices'][0]['message']['content']))

NOUS is not explicitly defined, so I cannot provide a precise list of services provided. However, if you are referring to a company or organization called NOUS, it would be helpful to have more context or the full name of the organization to provide accurate information. Alternatively, you can visit their official website or contact them directly to inquire about their services.

Then we see something even worse than `"I don't know"` — hallucinations. Clearly augmenting our queries with additional context can make a huge difference to the performance of our system.

Great, we've seen how to augment GPT-4 with semantic search to allow us to answer LangChain specific queries.

Once you're finished, we delete the index to save resources.

In [41]:
#pinecone.delete_index(index_name)